In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# Les réseaux à convolutions

Ces réseaux de neurones comprennent deux nouveaux types de couches :
    
- les couches de convolutions
- les couches de pooling

http://scs.ryerson.ca/~aharley/vis/conv/

# Les couches de convolution

La différence fondamentale entre une couche densément connectée et une couche de convolution est la suivante: 

- les couches denses apprennent des motifs globaux dans leur espace de fonctions en entrée (par exemple, pour un chiffre MNIST, des motifs impliquant tous les pixels), tandis que les couches de convolution apprennent des motifs locaux.

Dans le cas des images, des motifs trouvés dans de petites fenêtres 2D des entrées. 

![image](./images/conv.jpg)


Cette caractéristique clé confère à convnets deux propriétés intéressantes:

- Les modèles qu’ils apprennent sont des invariants de traduction.

- Ils peuvent apprendre les hiérarchies spatiales des modèles

Les convolutions sont définies par deux paramètres clés:

- Taille des patchs extraits des entrées

- Profondeur de la carte de caractéristiques en sortie

![image](./images/conv2.jpg)

# Les couches de pooling

Chaque couche MaxPooling2D permet de réduire la taille des cartes de caractéristiaues. 

C’est le rôle du pooling : sous-échantillonner de manière agressive les cartes de features.

# Un cas concret

Nous allons utiliser notre réseau convnet pour classer les chiffres MNIST.

Les 6 lignes de code ci-dessous vous montrent à quoi ressemble un convnet de base. 


C'est une pile de couches `Conv2D` et` MaxPooling2D`


Il est important de noter qu'un convnet prend en entrée des tenseurs de la forme `(image_height, image_width, image_channels)` 


Dans notre cas, nous allons configurer notre convnet pour traiter des entrées de taille `(28, 28, 1)`, qui est le format des images MNIST. 


Nous faisons cela via l'argument `input_shape = (28, 28, 1)`

In [1]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Voici son architecture :

In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Vous pouvez voir que la sortie de chaque couche `Conv2D` et` MaxPooling2D` est un tenseur 3D de forme `(hauteur, largeur, canaux)`. 


Les dimensions en largeur et en hauteur ont tendance à diminuer à mesure que nous progressons dans le réseau. Le nombre de canaux est contrôlé par le premier argument passé aux couches `Conv2D` (par exemple 32 ou 64).


Nous devons donc aplatir nos sorties 3D sur 1D, puis ajouter quelques couches `Dense` au-dessus:

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

Maintenant, entraînons notre réseau sur les chiffres du MNIST

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

 9536/10000 [===========================>..] - ETA: 0s

In [9]:
test_acc

0.99129999999999996